#### Information
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

#### Features Descriptions
1. **InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 
2. **StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
3. **Description**: Product (item) name. Nominal.
4. **Quantity**: The quantities of each product (item) per transaction. Numeric.	
5. **InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated.
6. **UnitPrice**: Unit price. Numeric, Product price per unit in sterling.
7. **CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
8. **Country**: Country name. Nominal, the name of the country where each customer resides.

## Preparing Environment

In [1]:
import pandas as pd 
import numpy as np
import datetime as dt 
from IPython.display import display


pd.set_option('display.max_columns', None)

data_uncleaned=pd.read_excel("../Data/Online Retail.xlsx")
data=data_uncleaned.copy()

## Exploring Data

### General Exploration

In [2]:
#exploring the shape of dataset and viewing first 10 rows of dataset

print("Shape of data: ", data.shape,"\n")
display(data.head(10))

Shape of data:  (541909, 8) 



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [3]:
# Checking data type of each feature and presence of any null values
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


None

Columns 'Description' and 'CustomerID' contain null values

Also, column "CustomerID" is detected as 'float64' data type when it should actually be object type

In [4]:
# checking the number of null values
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
# Checking unique values
data.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

According to data description, 'StockCode' is the code assigned to a unique product and 'Description' is the name of the product. So, in theory, number of unique values in 'StockCode' and 'Description' should be the same. But there seem to be more unique values in 'Description' as compared to 'StockCode' which could be the result of some kind of error or mistake that is worth investigating.

In [6]:
# Exploring numerical columns in dataset
display(data.describe())

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


**NOTE**: Ignore 'CustomerID' column because it is wrongly detected as float64 instead of object

We can observe that 'min' minimum value for 'Quantity' and 'UnitPrice' is a negative number. This could be the result of canceled or return orders but regardless it is worth investigating further.

In [7]:
#Checking for dubplicates
display(data.duplicated().sum())

5268

We can clearly observe that there are a lot of duplicate values in the dataset and these are needed to be removed in order to clean the data to aensure they don't affect the analysis of the data.

### Exploring 'Quantity' column

In [8]:
# Exploring negative quantity
display(data[data['Quantity']<0].head(10))
display(data[data['Quantity']<0].tail(10))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897.0,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540141,C581468,21314,SMALL GLASS HEART TRINKET POT,-10,2011-12-08 19:26:00,2.10,13599.0,United Kingdom
540142,C581468,22098,BOUDOIR SQUARE TISSUE BOX,-12,2011-12-08 19:26:00,0.39,13599.0,United Kingdom
540176,C581470,23084,RABBIT NIGHT LIGHT,-4,2011-12-08 19:28:00,2.08,17924.0,United Kingdom
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom
540448,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397.0,United Kingdom
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom
541717,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [9]:
print("Number of entries of canceled orders: ",len(data[data['InvoiceNo'].str.startswith('C')==True].index))
print("Number of entries of orders with negative quantity: ",len(data[data['Quantity']<=0]))

Number of entries of canceled orders:  9288
Number of entries of orders with negative quantity:  10624


As expected, most of the negative values in 'Quantity' arise as a result of cancelled orders or discounts etc. It might be worth keeping a seperate record of all these orders to study cancelled orders or to understand total profit and loss. However, there seem to be more entries with negative quantity than cancelled orders

In [10]:
display(data[(data['Quantity']<0) & (data['InvoiceNo'].str.contains('C', na=False)==False)].head(10))

display(data[(data['Quantity']<0) & (data['InvoiceNo'].str.contains('C', na=False)==False)].tail(10))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7192,537000,21414,NaN,-22,2010-12-03 15:32:00,0.0,NaN,United Kingdom
7193,537001,21653,NaN,-6,2010-12-03 15:33:00,0.0,NaN,United Kingdom
7195,537003,85126,NaN,-2,2010-12-03 15:33:00,0.0,NaN,United Kingdom
7196,537004,21814,NaN,-30,2010-12-03 15:34:00,0.0,NaN,United Kingdom
7197,537005,21692,NaN,-70,2010-12-03 15:35:00,0.0,NaN,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
535327,581204,85104,????damages????,-355,2011-12-07 18:32:00,0.0,NaN,United Kingdom
535328,581205,20893,damages,-55,2011-12-07 18:32:00,0.0,NaN,United Kingdom
535329,581206,21693,mixed up,-87,2011-12-07 18:34:00,0.0,NaN,United Kingdom
535330,581207,21688,mixed up,-337,2011-12-07 18:34:00,0.0,NaN,United Kingdom
535331,581208,72801C,check,-10,2011-12-07 18:35:00,0.0,NaN,United Kingdom
535333,581210,23395,check,-26,2011-12-07 18:36:00,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,2011-12-07 18:38:00,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,NaN,United Kingdom
538919,581422,23169,smashed,-235,2011-12-08 15:24:00,0.0,NaN,United Kingdom


On checking the orders that were not cancelled but still had negative values for 'Quantity', we can observe that they were lost, mixed up, damaged etc. In other words, they caused a loss to the organisation/company.

It might be worth keeping a seperate record of all such orders since it might help to analyse profit and loss based on these transactions

### Exploring 'UnitPrice' Column

In [11]:
# Exploring negative unit proce
display(data[data['UnitPrice']<0].head())

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


The negative unit price seem to be debt payments that occured with stock code = 'B' 

In [12]:
display(data[data['StockCode']=='B'].head())

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


We can observe that there was another transaction with same stock code and it was the same amount but it was not negative. This could be typing error



### Exploring 'Description' Column

In [13]:
display(len(data['Description'].unique()))

display(data['Description'].unique())

4224

array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ..., 'lost',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], dtype=object)

We can notice that there are 4224 unique values in Desscription column but as we noticed before, some of these values don't contain product name, but rather contain additional information like if the order went missing , was damaged , wet etc.

The true product names are all stored in upper case and other information is either stored as title case or lower case

In [14]:
#creating a new dataframe to hold the list of all different unique values in 'Description' column
desc_unique=pd.DataFrame(data['Description'].unique().astype(str), columns=['Desc_unique'])


In [15]:
#viewing all uppercase descrption (majority are product names)
display(desc_unique[desc_unique['Desc_unique'].str.isupper()==True])

#viewing all lowercase description
display(desc_unique[desc_unique['Desc_unique'].str.islower()==True])

#viewing all titlecase description
display(desc_unique[desc_unique['Desc_unique'].str.istitle()==True])

#viewing all other types of case in description
display(desc_unique[(desc_unique['Desc_unique'].str.isupper()==False)&(desc_unique['Desc_unique'].str.islower()==False)&(desc_unique['Desc_unique'].str.istitle()==False)])

,Desc_unique
0,WHITE HANGING HEART T-LIGHT HOLDER
1,WHITE METAL LANTERN
2,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART.
...,...
4210,SET 10 CARDS SNOWY ROBIN 17099
4212,SET 10 CARDS SWIRLY XMAS TREE 17104
4216,"LETTER ""U"" BLING KEY RING"
4222,CREAM HANGING HEART T-LIGHT HOLDER


,Desc_unique
395,nan
1740,amazon
2155,check
2157,damages
2394,faulty
...,...
4217,wet
4218,wet boxes
4219,????damages????
4220,mixed up


,Desc_unique
111,Discount
1108,Manual
1537,Bank Charges
2373,*Boombox Ipod Classic
2722,Dotcomgiftshop Gift Voucher £40.00
2727,Found
2754,Dotcomgiftshop Gift Voucher £50.00
2781,Dotcomgiftshop Gift Voucher £30.00
2782,Dotcomgiftshop Gift Voucher £20.00
2910,Dotcom


,Desc_unique
326,BAG 500g SWIRLY MARBLES
550,POLYESTER FILLER PAD 45x45cm
1037,BAG 125g SWIRLY MARBLES
1038,BAG 250g SWIRLY MARBLES
1052,POLYESTER FILLER PAD 45x30cm
1053,POLYESTER FILLER PAD 40x40cm
1061,FRENCH BLUE METAL DOOR SIGN No
1187,Dr. Jam's Arouzer Stress Ball
1367,3 TRADITIONAl BISCUIT CUTTERS SET
1582,NUMBER TILE COTTAGE GARDEN No


We can observe that there are a lot of non-product name entries in the 'Description' column and there also seem to be some typing errors in it as well. 

### Exploring 'StockCode' Column

In [16]:
#total number of different products based on StockCode
display(len(data['StockCode'].unique()))

display(data['StockCode'].unique())

4070

array(['85123A', 71053, '84406B', ..., '90214U', '47591b', 23843],
      dtype=object)

We can observe that there are a total of 4070 unique entries in 'StockCode' column and most of them are either purely numeric or purely alpha-numeric values

In [17]:
#creating a dataframe of all unique values in 'StockCode' column
stockcode_unique=pd.DataFrame(data['StockCode'].unique().astype(str), columns=['StockCode_unique'])

In [18]:
#viewing all purely numeric stock codes
display(stockcode_unique[stockcode_unique['StockCode_unique'].str.isnumeric()==True])

#viewing all purely alpha numeric stock codes
display(stockcode_unique[(stockcode_unique['StockCode_unique'].str.isalnum()==True)&(stockcode_unique['StockCode_unique'].str.isalpha()==False)&(stockcode_unique['StockCode_unique'].str.isnumeric()==False)])

#viewing all purely alphabetical stock codes
display(stockcode_unique[stockcode_unique['StockCode_unique'].str.isalpha()==True])

#viewing all other stock codes
display(stockcode_unique[(stockcode_unique['StockCode_unique'].str.isalnum()==False)&(stockcode_unique['StockCode_unique'].str.isalpha()==False)&(stockcode_unique['StockCode_unique'].str.isnumeric()==False)])

,StockCode_unique
1,71053
5,22752
6,21730
7,22633
8,22632
...,...
4062,23562
4063,23561
4064,23609
4066,23617


,StockCode_unique
0,85123A
2,84406B
3,84029G
4,84029E
55,82494L
...,...
4055,84971l
4056,85034b
4065,85179a
4067,90214U


,StockCode_unique
45,POST
111,D
952,DOT
1115,M
2242,S
2245,AMAZONFEE
2791,m
3078,DCGSSBOY
3079,DCGSSGIRL
3328,PADS


,StockCode_unique
1542,BANK CHARGES
2774,gift_0001_40
2815,gift_0001_50
2842,gift_0001_30
2843,gift_0001_20
3197,gift_0001_10


We can observe that there are a lot of entries in 'StockCode' which indicates that there is data about non-sales transactions. This means we need to take care of it during the cleaning phase and remove it since the primary objective of analysis on this data is going to be to analyse the sales of different projects and purcheses made my different customers. Hence, any non-sales data is irrelevant to the project.

### Exploring 'Country' column

In [19]:
#checking 'Country' column to look for any errors
data['Country'].value_counts().sort_index()

Australia                 1259
Austria                    401
Bahrain                     19
Belgium                   2069
Brazil                      32
Canada                     151
Channel Islands            758
Cyprus                     622
Czech Republic              30
Denmark                    389
EIRE                      8196
European Community          61
Finland                    695
France                    8557
Germany                   9495
Greece                     146
Hong Kong                  288
Iceland                    182
Israel                     297
Italy                      803
Japan                      358
Lebanon                     45
Lithuania                   35
Malta                      127
Netherlands               2371
Norway                    1086
Poland                     341
Portugal                  1519
RSA                         58
Saudi Arabia                10
Singapore                  229
Spain                     2533
Sweden  

There don't seem to be any errors in the values of 'Country' column. However, there seem to be a value named 'unspecified'.This means that there are some rows have missing values for 'Country' column and instead of NULL they store the value 'Unspecified'

## Data Cleaning

Now we have explored the data and are familiar with it, we can now start cleaning it based on what we have learned from exploring it

For cleaning the data we need to do the following things:
1. **fixing Data type**: We observed during data exploring that the column 'CustomerID' is of Float64 data type when it should actually be just string type and there are other columsn that are of datatype 'Object' and are neede to be converted into string

2. **Delete Duplicate Data**: The duplicate rows in the dataset needs to be removed to clean the dataset for analysis. We can do this using duplicate() function of pandas library for DataFrame objects.

3. **Remove non-sales transactions data**: From exploring data we observed that a lot of data in the dataset is non-sales related transactional data, for example - amazon fees, bank charges, gifts, etc. All this data is irrelevant because data needs to be cleaned for analysis of sales of different products. One way to doing this would be to use StockCodes column. The StockCodes column mostly consists of product id of different products, and these product ids are either purely numeric values or are alpha numeric values and they are of length 5 or more, all other enteries are for non-sales transactions.

4. **Make a new dataset for data of profit and non profit transactions**: While exploring 'Description' column, we observed a lot of irregularities and missing values in the 'Description' column. We could also observe that in almost all of these cases, the 'Quantity' column had a negative value meaning these were a loss for the company. We will make a seperate dataset for such data and clean it as well and save it seperately since it might be needed for evaluation of losses due to different reasons. 

5. **Dealing with missing values**: During exploration, it was observed that there were a lot of missing values for columns 'Descrption', 'CustomerID' and 'Country'(they were marked as "unspecified" in country column).
        
     * **For 'Description'**: For this column, if there was any cancelled order or any non cancelation order with a positive 'Quantity' value with missing descrption then it can be corrected by putting the value of any non null description with same stock code. For non cancelation orders with negative 'Quantity' value with missing description, they will be marked as "damaged" and if their description value is only "?" or "??" or "???", they will be marked as "missing"
        
     * **For 'CustomerID'**: For this column, almost all missing values are for non-sales transacations and if they are missing in sales transaction data, those rows of data will not be removed since it might affect the analysis of the data
        
     * **For 'Country'**: Rows with "unspecified" values for this columns will not be removed as well but shall be taken note of during data analysis which will not be the part of this project.


### 1. Fixing Data types

Now we will change the data types of the columns to make sure they are of correct data type. In order to achieve that we will convert all columns with datatype as "object" to string because columns with object datatype can hold strings, floats, integers and any other datatype values in them and we need to make sure they are all of same datatype, that is string datatype, to keep the dataset consistent.

In [20]:
#changing data types to string
data['InvoiceNo']=data['InvoiceNo'].astype(str) 

data['StockCode']=data['StockCode'].astype(str)

data['Description']=data['Description'].astype(str)

data['CustomerID']=data['CustomerID'].astype(str)

data['Country']=data['Country'].astype(str)

### 2. Deleting Dubplicates

We will delete dublicate values from the dataset using duplicate() function.

In [21]:
data = data[~data.duplicated()]

### 3. Removing non-sales Transactional data

We will create a seperate dataset to store all sales related transactional data. We will do this by using only numeric and alpha numeric StockCode values since all other StockCode values are for non sales transactions data.

In [22]:
#creating a list of purely numeric and alpha-Numeric stockcodes
alphaNum_stockcode=sum(stockcode_unique[(stockcode_unique['StockCode_unique'].str.isalnum()==True)&(stockcode_unique['StockCode_unique'].str.isalpha()==False)&(stockcode_unique['StockCode_unique'].str.len()>=4)].values.tolist(), [])

#creating a new dataset for sales data
data_sales=pd.DataFrame(columns=data.columns.tolist())

#iterating through each stock code and adding it to the new dataset for sales data
for x in alphaNum_stockcode:
    data_sales=pd.concat([data_sales, data[data['StockCode']==x]], axis=0)

In [23]:
display(data_sales)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom
262,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.55,13408.0,United Kingdom
...,...,...,...,...,...,...,...,...
512588,579512,23617,SET 10 CARDS SWIRLY XMAS TREE 17104,2,2011-11-29 16:47:00,2.91,nan,United Kingdom
527065,580691,90214U,"LETTER ""U"" BLING KEY RING",12,2011-12-05 15:48:00,0.29,13790.0,United Kingdom
537224,581238,47591b,SCOTTIES CHILDRENS APRON,1,2011-12-08 10:53:00,4.13,nan,United Kingdom
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom


### 4. Making a new dataset for data of profit and non profit transactions

We will create seperate datasets for profit adn non profit sales data. We will do this using the value sin "Quantity" column. All negative values in Quantity column indicates the non-profit transactions and all positive values indicates the profit data.

In [24]:
#creating a new dataset for profitable sales data
data_sales_profit=data_sales[data_sales['Quantity']>0]

display(data_sales_profit)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom
262,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.55,13408.0,United Kingdom
...,...,...,...,...,...,...,...,...
509369,579297,85179a,GREEN BITTY LIGHT CHAIN,1,2011-11-29 11:23:00,2.46,nan,United Kingdom
512588,579512,23617,SET 10 CARDS SWIRLY XMAS TREE 17104,2,2011-11-29 16:47:00,2.91,nan,United Kingdom
527065,580691,90214U,"LETTER ""U"" BLING KEY RING",12,2011-12-05 15:48:00,0.29,13790.0,United Kingdom
537224,581238,47591b,SCOTTIES CHILDRENS APRON,1,2011-12-08 10:53:00,4.13,nan,United Kingdom


In [25]:
#creating new dataset for non-profit sales data.
data_sales_nonProfit=data_sales[data_sales['Quantity']<0]

display(data_sales_nonProfit)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14529,C537602,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1,2010-12-07 12:45:00,2.55,17511.0,United Kingdom
24911,C538350,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1,2010-12-10 15:01:00,2.55,13798.0,United Kingdom
29949,C538834,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1,2010-12-14 13:06:00,2.95,14415.0,United Kingdom
34292,C539329,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-500,2010-12-17 09:37:00,2.55,16013.0,United Kingdom
38444,C539576,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-24,2010-12-20 12:25:00,2.95,14911.0,EIRE
...,...,...,...,...,...,...,...,...
431382,573597,79341,"Unsaleable, destroyed.",-905,2011-10-31 15:18:00,0.0,nan,United Kingdom
431383,573598,79342B,"Unsaleable, destroyed.",-1128,2011-10-31 15:18:00,0.0,nan,United Kingdom
515699,C579783,23597,PAPER BUNTING PAISLEY PARK,-6,2011-11-30 15:21:00,2.95,17727.0,United Kingdom
516138,C579869,23428,IVORY RETRO KITCHEN WALL CLOCK,-2,2011-11-30 16:44:00,8.15,14189.0,United Kingdom


### 5. Missing values in 'Description' Column

We saw earlier that there are missing values which are marked as either NULL of marked as "?", "??" or "???" in 'Description' column but these can be corrected by using values from other entries with same stock code.

**NOTE**: We are not removing any rows which contain null values because that wil have a huge impact on the final analysis of the data hence we will keep all rows containaing null or missing values for any column.

In [26]:
print("NULL values in profit sales data: ",data_sales_profit[data_sales_profit['Description']=='nan'].shape[0])
print("NULL values in non-profit sales data: ",data_sales_nonProfit[data_sales_nonProfit['Description']=='nan'].shape[0])

NULL values in profit sales data:  584
NULL values in non-profit sales data:  862


In [27]:
print("Missing values in profit sales data: ")
display(data_sales_profit[data_sales_profit['Description']=='?'])

Missing values in profit sales data: 


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
220843,556231,85123A,?,4000,2011-06-09 15:04:00,0.0,nan,United Kingdom
38261,539494,21479,?,752,2010-12-20 10:36:00,0.0,nan,United Kingdom
282882,561665,22171,?,142,2011-07-28 16:55:00,0.0,nan,United Kingdom
115807,546139,84988,?,3000,2011-03-09 16:35:00,0.0,nan,United Kingdom
421093,572920,72803A,?,117,2011-10-26 16:52:00,0.0,nan,United Kingdom
323315,565288,23135,?,101,2011-09-02 11:43:00,0.0,nan,United Kingdom


In [28]:
print("Missing values in non-profit sales data: ")
display(data_sales_nonProfit[data_sales_nonProfit['Description']=='?'])
display(data_sales_nonProfit[data_sales_nonProfit['Description']=='??'])
display(data_sales_nonProfit[data_sales_nonProfit['Description']=='???'])

Missing values in non-profit sales data: 


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
128467,547339,84406B,?,-450,2011-03-22 11:51:00,0.0,nan,United Kingdom
50806,540558,21258,?,-29,2011-01-10 10:04:00,0.0,nan,United Kingdom
94765,544410,84997C,?,-741,2011-02-18 13:54:00,0.0,nan,United Kingdom
258540,559655,21166,?,-124,2011-07-11 12:57:00,0.0,nan,United Kingdom
68920,541882,48185,?,-57,2011-01-24 10:54:00,0.0,nan,United Kingdom
43662,540100,22837,?,-106,2011-01-04 16:53:00,0.0,nan,United Kingdom
115814,546148,85049D,?,-100,2011-03-09 16:43:00,0.0,nan,United Kingdom
395168,571024,22812,?,-270,2011-10-13 12:18:00,0.0,nan,United Kingdom
384187,570110,22570,?,-74,2011-10-07 12:27:00,0.0,nan,United Kingdom
384222,570112,22569,?,-84,2011-10-07 12:29:00,0.0,nan,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
337534,566449,20856,??,-140,2011-09-12 16:07:00,0.0,nan,United Kingdom
418840,572749,22572,??,-335,2011-10-25 16:52:00,0.0,nan,United Kingdom
340683,566712,22030,??,-635,2011-09-14 12:04:00,0.0,nan,United Kingdom
455407,575615,82582,??,-130,2011-11-10 12:51:00,0.0,nan,United Kingdom
390852,570599,84596G,??,-365,2011-10-11 12:00:00,0.0,nan,United Kingdom
418130,572702,37479P,??,-184,2011-10-25 14:33:00,0.0,nan,United Kingdom
381687,569886,90191,??,-60,2011-10-06 15:24:00,0.0,nan,United Kingdom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
524370,580547,21201,???,-390,2011-12-05 09:29:00,0.0,nan,United Kingdom


We can note that there are 584 NULL values in the profit dataset and 862 NULL values in the non-profit dataset.

Moreover the missing values are marked as "?" or "??" or "???" in both the datasets.


* **For profit sales data**, we will replace NULL and "?" with product names by matching the stock codes.
* **For non-profit sales data**, we will replace NULL "?", "??", "???" with the product names by matching the Stockcodes if the order was for cancelled orders (marked with a 'C' in the invoice number) and for others, we will replace them with "missing".

In [29]:
display(data_sales_profit[data_sales_profit['Description']=='?'].loc[:,'StockCode'])
display(data_sales_profit[data_sales_profit['Description']=='?'].loc[:,'StockCode'].values)

220843    85123A
38261      21479
282882     22171
115807     84988
421093    72803A
323315     23135
Name: StockCode, dtype: object

array(['85123A', '21479', '22171', '84988', '72803A', '23135'],
      dtype=object)

In [30]:
for stockcode in np.unique(data_sales_profit[data_sales_profit['Description']=='?']['StockCode'].values):
    print(data_sales_profit[data_sales_profit['StockCode']==stockcode]['Description'].unique()[0])

WHITE SKULL HOT WATER BOTTLE 
3 HOOK PHOTO SHELF ANTIQUE WHITE
SMALL ZINC HEART WALL ORGANISER
ROSE SCENT CANDLE JEWELLED DRAWER
SET OF 72 PINK HEART PAPER DOILIES
WHITE HANGING HEART T-LIGHT HOLDER


In [31]:
for stockcode in np.unique(data_sales_profit[data_sales_profit['Description']=='nan']['StockCode'].values):
    # if (data_sales_profit[data_sales_profit['StockCode']==stockcode]['Description'].unique()[0] != 'nan'):
    #     print(data_sales_profit[data_sales_profit['StockCode']==stockcode]['Description'].unique()[0])
    if 'nan' in (data_sales_profit[data_sales_profit['StockCode']==stockcode]['Description'].unique()):
        if(len(data_sales_profit[data_sales_profit['StockCode']==stockcode]['Description'].unique())>1):
            if (data_sales_profit[data_sales_profit['StockCode']==stockcode]['Description'].unique()[0])=='nan':
                print(data_sales_profit[data_sales_profit['StockCode']==stockcode]['Description'].unique()[1])
            else:
                print(data_sales_profit[data_sales_profit['StockCode']==stockcode]['Description'].unique()[0])
        else:
            print("No description for stock code = ", stockcode)

INFLATABLE POLITICAL GLOBE 
GROOVY CACTUS INFLATABLE
FAN BLACK FRAME 
BLUE POLKADOT GARDEN PARASOL
FAIRY CAKE DESIGN UMBRELLA
MEDIUM CHINESE STYLE SCISSOR
MINI HIGHLIGHTER PENS
TEATIME PEN CASE & PENS
TEATIME GEL PENS ASST
THE KING GIFT BAG
PINK STRAWBERRY HANDBAG 
PINK HEART RED HANDBAG
BLUE HOLE PUNCH
ORIGAMI VANILLA INCENSE CONES
LAVENDER INCENSE IN TIN
ASS COL LARGE SAND FROG P'WEIGHT
PORCELAIN BUTTERFLY OIL BURNER
TROPICAL HOLIDAY PURSE 
STRAWBERRY DREAM CHILDS UMBRELLA
PINK PADDED MOBILE
JUMBO BAG OWLS
PARTY FOOD SHOPPER BAG
POSY CANDY BAG
SILVER MINI TAPE MEASURE 
KENSINGTON COFFEE SET
GARDEN PATH POCKET BOOK
GREEN FERN NOTEBOOK
GARDEN PATH NOTEBOOK 
CLEAR MILKSHAKE GLASS
LARGE PINK GLASS SUNDAE DISH
GOLDIE LOOKING MIRROR
No description for stock code =  20849
ZINC HEART LATTICE TRAY OVAL 
SET/9 CHRISTMAS T-LIGHTS SCENTED 
BOX OF 9 PEBBLE CANDLES
FROG CANDLE
No description for stock code =  20950
PORCELAIN T-LIGHT HOLDERS ASSORTED
POLYESTER FILLER PAD 60x40cm
SANDWICH BATH SPONG

In [32]:
for invoice in (data_sales_nonProfit[data_sales_nonProfit['InvoiceNo'].str.startswith('C')==False]['InvoiceNo'].values):
    if 'nan' in (data_sales_nonProfit[data_sales_nonProfit['InvoiceNo']==invoice]['Description'].values):
        temp_code=data_sales_nonProfit[data_sales_nonProfit['InvoiceNo']==invoice]['StockCode'].values
        temp_code=temp_code[0]
        if(len(data_sales_profit[data_sales_profit['StockCode']==temp_code]['Description'].unique())>1):
            if (data_sales_profit[data_sales_profit['StockCode']==temp_code]['Description'].unique()[0]=='nan'):
                print(data_sales_profit[data_sales_profit['StockCode']==temp_code]['Description'].unique()[1])
            else:
                print(data_sales_profit[data_sales_profit['StockCode']==temp_code]['Description'].unique()[0])
        else:
            print("No description for stock code = ", temp_code)
        


No description for stock code =  84406B
RED WOOLLY HOTTIE WHITE HEART.
RED WOOLLY HOTTIE WHITE HEART.
No description for stock code =  22633
HAND WARMER RED POLKA DOT
RECIPE BOX WITH METAL HEART
ALARM CLOCK BAKELIKE GREEN
INFLATABLE POLITICAL GLOBE 
WOOD 2 DRAWER CABINET WHITE FINISH
WOODEN FRAME ANTIQUE WHITE 
WOODEN FRAME ANTIQUE WHITE 
HOT WATER BOTTLE TEA AND SYMPATHY
HOT WATER BOTTLE TEA AND SYMPATHY
HOT WATER BOTTLE TEA AND SYMPATHY
JUMBO BAG CHARLIE AND LOLA TOYS
JUMBO BAG CHARLIE AND LOLA TOYS
RETROSPOT TEA SET CERAMIC 11 PC 
No description for stock code =  84854
No description for stock code =  82567
No description for stock code =  21169
GIN + TONIC DIET METAL SIGN
PINK BREAKFAST CUP AND SAUCER 
GUMBALL MONOCHROME COAT RACK
No description for stock code =  22926
3 TIER CAKE TIN RED AND CREAM
No description for stock code =  35004C
No description for stock code =  35004C
HANGING MEDINA LANTERN SMALL
HEART OF WICKER SMALL
HEART OF WICKER LARGE
ORGANISER WOOD ANTIQUE WHITE 
ORG